<a href="https://colab.research.google.com/github/abulhasanat/MachineLearning/blob/master/Web_classification_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#I'm using my googledrive to access and store models and data. Any local filesystem can be used instead off
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# This library will be used by boilerpipe
!pip install JPype1

In [3]:
# I'm using boilerpipe to extract noisefree text for classification from webpage
!git clone https://github.com/misja/python-boilerpipe.git
%cd /content/python-boilerpipe
!sudo python setup.py install

fatal: destination path 'python-boilerpipe' already exists and is not an empty directory.
/content/python-boilerpipe
/usr/lib/python3.6/distutils/dist.py:261: UserWarning: Unknown distribution option: 'install_requires'
  warnings.warn(msg)
running install
running build
running build_py
running install_lib
running install_egg_info
Removing /usr/local/lib/python3.6/dist-packages/boilerpipe-1.3.0.0.egg-info
Writing /usr/local/lib/python3.6/dist-packages/boilerpipe-1.3.0.0.egg-info


In [4]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import numpy as np
from urllib.parse import urlparse

In [6]:
import requests
requests.packages.urllib3.disable_warnings()

import random
import math
import string
import re
import numpy as np
import nltk
nltk.download('punkt')
import networkx as nx
import json

import collections
from boilerpipe.extract import Extractor
import shutil #Delete folder subtree

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.6/dist-packages/jpype/_core.py:209: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [0]:
from pathlib import Path
from typing import *

import torch
import torch.optim as optim

In [0]:
from fastai import *
from fastai.vision import *
from fastai.text import *
from fastai.callbacks import *

In [9]:
#Installing Nude classifier
!pip install git+https://github.com/abulhasanat/NudeNet

  Cloning https://github.com/abulhasanat/NudeNet to /tmp/pip-req-build-sje0zp3g
  Running command git clone -q https://github.com/abulhasanat/NudeNet /tmp/pip-req-build-sje0zp3g
  Created wheel for NudeNet: filename=NudeNet-1.0.5-cp36-none-any.whl size=17656 sha256=35f8a3ea9069a0d29824f10d01c8767c7a62140f0183e3fab4634bb3cebb111e
  Stored in directory: /tmp/pip-ephem-wheel-cache-ral8oomy/wheels/45/0e/c0/055167fe95de5a2ed518d76f73da36b94cc21de561c770e6fc
Successfully built NudeNet


In [10]:
from nudenet import NudeClassifier

Using TensorFlow backend.


In [11]:
classifier = NudeClassifier('/content/drive/My Drive/Data/Nudenet/classifier_model')















Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [12]:
%%bash
pip install pytorch-pretrained-bert

In [0]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)

config = Config(
    testing=True,
    bert_model_name="bert-base-uncased",
    max_lr=3e-5,
    epochs=4,
    use_fp16=False,
    bs=32,
    discriminative=False,
    max_seq_len=256,
)

In [0]:
from pytorch_pretrained_bert import BertTokenizer
bert_tok = BertTokenizer.from_pretrained(
    config.bert_model_name,
)

In [0]:
def _join_texts(texts:Collection[str], mark_fields:bool=False, sos_token:Optional[str]=BOS):
    """Borrowed from fast.ai source"""
    if not isinstance(texts, np.ndarray): texts = np.array(texts)
    if is1d(texts): texts = texts[:,None]
    df = pd.DataFrame({i:texts[:,i] for i in range(texts.shape[1])})
    text_col = f'{FLD} {1} ' + df[0].astype(str) if mark_fields else df[0].astype(str)
    if sos_token is not None: text_col = f"{sos_token} " + text_col
    for i in range(1,len(df.columns)):
        #text_col += (f' {FLD} {i+1} ' if mark_fields else ' ') + df[i]
        text_col += (f' {FLD} {i+1} ' if mark_fields else ' ') + df[i].astype(str)
    return text_col.values

In [0]:
class FastAiBertTokenizer(BaseTokenizer):
    """Wrapper around BertTokenizer to be compatible with fast.ai"""
    def __init__(self, tokenizer: BertTokenizer, max_seq_len: int=128, **kwargs):
        self._pretrained_tokenizer = tokenizer
        self.max_seq_len = max_seq_len

    def __call__(self, *args, **kwargs):
        return self

    def tokenizer(self, t:str) -> List[str]:
        """Limits the maximum sequence length"""
        return ["[CLS]"] + self._pretrained_tokenizer.tokenize(t)[:self.max_seq_len - 2] + ["[SEP]"]

In [0]:
class BertTokenizeProcessor(TokenizeProcessor):
    def __init__(self, tokenizer):
        super().__init__(tokenizer=tokenizer, include_bos=False, include_eos=False)

class BertNumericalizeProcessor(NumericalizeProcessor):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, vocab=Vocab(list(bert_tok.vocab.keys())), **kwargs)

def get_bert_processor(tokenizer:Tokenizer=None, vocab:Vocab=None):
    """
    Constructing preprocessors for BERT
    We remove sos/eos tokens since we add that ourselves in the tokenizer.
    We also use a custom vocabulary to match the numericalization with the original BERT model.
    """
    return [BertTokenizeProcessor(tokenizer=tokenizer),NumericalizeProcessor(vocab=vocab)]

In [0]:
def getUA():
    uastrings = ["Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1500.72 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10) AppleWebKit/600.1.25 (KHTML, like Gecko) Version/8.0 Safari/600.1.25",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",\
                "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/600.1.17 (KHTML, like Gecko) Version/7.1 Safari/537.85.10",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko",\
                "Mozilla/5.0 (Windows NT 6.3; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.104 Safari/537.36"\
                ]

    return random.choice(uastrings)

def prepare_sentences(text):
    text=text.replace('\n',' ')
    # Clean characters
    text = "".join([t for t in text if t.isalnum() or t in string.punctuation or t == ' '])
    
    # Get rid of whitespace characters
    sentences = nltk.sent_tokenize(text)
    text = " ".join([s.strip() for s in sentences])
    
    # Fix puctuation spacing
    text = re.sub(r"(\w{2,})([\.\!\?]+)(\w)", r"\1\2 \3", text)
    text = re.sub(' +', ' ', text)
    return text

def find_children(n1, n2):
    
    children1 = [c for c in n1.children]
    children2 = [c for c in n2.children]
    
    for i, children in enumerate([children1,children2]):
        level = 1

        while children:
            lchildren = children
            children = []
            for ch in lchildren:
                if not ch.name:
                    continue
                if ch == n1 or ch == n2:
                    if i == 0:
                        return len( nltk.sent_tokenize(n2.get_text()) ) * ( 1/math.exp(level) )
                    else:
                        return len( nltk.sent_tokenize(n1.get_text()) ) * ( 1/math.exp(level) )
                                        
                children.extend(ch.children)

            level += 1
            
    return 0

def extract_html(url, timeout=10):
    headers = {'user-agent': getUA()}
    r = requests.get(url, headers = headers, verify=False, timeout=timeout)
    html = r.content
    return html

def tokenize_url(url:str):   
    url=re.sub("(\W|_)+"," ",url)
    return url

In [0]:
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("<!--?.*?-->","",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

In [0]:
def get_title(soup):
    try:
        for i in list(soup.find_all('title')):
            return(i.get_text().replace("\n"," ").replace("\t"," ").strip())
    except:
        pass

def list2string(x):
    s=""
    try:
        for i in x:
            for j in i:
                if j!=None:
                    s = s + j+" "
    except:
        pass
    return (s)

def get_metadata_property(soup):
    try:
        x=[i.get('property') for i in soup.findAll('meta')]
        v=[i.get('content') for i in soup.findAll('meta')]
        z=np.c_[x,v]
        return list2string(z)
    except:
        pass

def get_metadata_name(soup):
    try:
        y=[i.get('name') for i in soup.findAll('meta')]
        v=[i.get('content') for i in soup.findAll('meta')]
        z=np.c_[y,v]
        return list2string(z)
    except:
        pass

def get_metadata_content(soup):
    try:
        v=[]
        v=[i.get('content') for i in soup.findAll('meta')]
    #     return list2string(v)
        return (v)
    except:
        pass

def get_text(soup):
    text=""
    try:
        for item in list(soup.find_all('p')):  #[:2]:
            text+=item.get_text()+" "
    except:
        pass
    return text
#     return (len(text.split())/COUNT)

In [0]:
def get_word_count(soup):
    text=""
    try:
        for item in list(soup.find_all('p')):  #[:2]:
            text+=item.get_text()+" "
    except:
        pass
    return len(text.split())

def get_para_count(soup):
    count=0
    try:
        for item in list(soup.find_all('p')):  #[:2]:
            count+=1
    except:
        pass
    return count

In [0]:
def get_internal_links(url_main,links):
    internal_url_list=[]
    try:
        for item in links:
            for url_parts in url_main.split('.')[:-1]:
                if (url_parts in item.lower()):
                    internal_url_list.append(item)
        internal_url_list=list(dict.fromkeys(internal_url_list))    
        return(len(internal_url_list))
    except:
        pass

def get_external_links(url_main,links):
    external_url_list=[]
    try:
        for item in links:
            count=0
            for url_parts in url_main.split('.')[:-1]:
                if (item.startswith(('http', 'ftp', 'www')) and (url_parts not in item.lower())):
                    count+=1
            if count==len(url_main.split('.')[:-1]):
                external_url_list.append(item)
        return(len(external_url_list))
    except:
        pass

In [0]:
def extract_features(url):
    # create a new Chrome session
    try:
        print(url)
        metadata_property=''
        metadata_name=''
        metadata_content=''
        title=''
        word_count=0
        sen_count=0
        internal_links=0
        external_links=0
    #         driver = webdriver.Chrome(executable_path=r'D:/chromedriver_win32/chromedriver.exe')
    #         driver.set_page_load_timeout(60)
    #         driver.get(url)
    #         html = driver.page_source
    #         driver.close()
    #         soup = BeautifulSoup(html, 'html.parser')  #Parse html code       
    #         body = soup.find('body')

        html= extract_html(url, 100)
        soup = BeautifulSoup(html, 'lxml')
        body = soup.find('body')
        remove = [s.extract() for s in body(["script", "style","iframe","noscript","nav","footer","header",\
                                             "svg", "h1","h2", "h3", "h4", "h5", "xml"])]
        text = ""
        boilerpipe_sentences=""
        boilerpipe_content = prepare_sentences(Extractor(extractor='CanolaExtractor', html=html).getText())
        boilerpipe_sentences = prepare_sentences(boilerpipe_content)
    #         print('Sentences:',len(boilerpipe_sentences))

#         p = '(?:http.*://)?(?:www.)?(?P<host>[^/ ]+).?(?P<port>[0-9]*).*'
        p = '(?:http.*://)?(?:www[0-9].|www.)?(?P<host>[^/ ]+).?(?P<port>[0-9]*).*'
        m = re.search(p,url)
        url_main = m.group('host') # 'www.abc.com'
        if len(soup)>0:
            metadata_property=get_metadata_property(soup)
            metadata_name=get_metadata_name(soup)
            metadata_content=get_metadata_content(soup)
            title=get_title(soup)
            
            word_count=len(nltk.word_tokenize(boilerpipe_sentences))
            
            sen_count=len(nltk.sent_tokenize(boilerpipe_sentences))       

            links = [ x['href'] for x in soup.find_all('a', href=True) ]
            if len(links)>0:
                internal_links=get_internal_links(url_main,links)
                external_links=get_external_links(url_main,links)

    #         return pd.Series([get_metadata_property(soup),get_metadata_name(soup),get_metadata_content(soup),
    #                       get_title(soup)])
    except:
        pass
    
    extract= pd.Series([metadata_property,metadata_name,metadata_content,
                          title,word_count,sen_count,internal_links,external_links])
    #         extract= pd.Series([get_metadata_property(soup),get_metadata_name(soup)])
    return extract

In [0]:
# test=pd.DataFrame(pd.Series([boilerpipe_sentences,0,0,0,0,0,0]),columns=['category','label_1',	'label_3',	'label_5',	'label_6',	'label_7',	'label_8'])

In [0]:
#Labels for webpage type
label_cols_classify=['Science', 'Society',  'Computers',  'Shopping',  'Business',  'Recreation',  'Sports',  'Health',  'Reference',  'Games',
 'Adult',  'Kids_and_Teens',  'Home',  'Arts',  'News']

label_cols_toxic = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

In [26]:
#Loading model to classify any obsence comments in the page
DATA_ROOT=Path('/content/drive/My Drive/Data/toxic_comments')
learn_toxic=load_learner(DATA_ROOT,'toxic_comments.pkl')
learn_toxic.to_fp32()

Learner(data=TextClasDataBunch;

Train: LabelList (0 items)
x: TextList

y: MultiCategoryList

Path: /content/drive/My Drive/Data/toxic_comments;

Valid: LabelList (0 items)
x: TextList

y: MultiCategoryList

Path: /content/drive/My Drive/Data/toxic_comments;

Test: None, model=BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
     

In [27]:
DATA_ROOT=Path('/content/drive/My Drive/Data/DMOZ')
# learner.load(file=DATA_ROOT/'Model_DMOZ(6_cat).pkl')
# learn = load_learner(DATA_ROOT, 'Model_DMOZ(6_cat).pkl')
# learn = load_learner(DATA_ROOT, 'Model_DMOZ(25_cat).pkl')
learn_classify = load_learner(DATA_ROOT, 'Model_DMOZ(all_cat_v1_2).pkl')
learn_classify.to_fp32()


Learner(data=TextClasDataBunch;

Train: LabelList (0 items)
x: TextList

y: MultiCategoryList

Path: /content/drive/My Drive/Data/DMOZ;

Valid: LabelList (0 items)
x: TextList

y: MultiCategoryList

Path: /content/drive/My Drive/Data/DMOZ;

Test: None, model=BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): 

In [0]:
# def get_text1(soup):
#     text=""
#     for item in list(soup.find_all('body')):  #[:2]:
#         text+=item.get_text()+" "
#     return text

In [29]:
site='https://www.curvyerotic.com/model/nora-rose//'
# site='https://www.wallmart.com/'
# site='https://www.foxnews.com/'
html= extract_html(site, 60)
soup = BeautifulSoup(html, 'lxml')
body = soup.find('body')
remove = [s.extract() for s in body(["script", "style","iframe","noscript","nav","footer","header",\
                                     "svg", "h1","h2", "h3", "h4", "h5", "xml"])]
# remove = [s.extract() for s in body(["script", "style","iframe","noscript","nav","footer","header",\
#                                      "svg", "xml"])]
boilerpipe_content = prepare_sentences(Extractor(extractor='CanolaExtractor', html=html).getText())
boilerpipe_sentences = prepare_sentences(boilerpipe_content)
# boilerpipe_sentences=get_text1(soup)
print(boilerpipe_sentences)

Porn Games Nora Rose Scarf Nude Muse Nora Rose knows posing with a scarf is true nude art, and I love it! To me when I go on Nude Muse, Im looking for real nude girls who like to shake things up. As you can see, Nora is well known for her hands on posing, and I hope you enjoy the show. Nora Rose Silk Robe Seduction Nude Muse There is something seriously seductive about Nora Rose, one of the Nude Muses most impressive model. What I mean by that is simply watching her pose in a silk robe is sexy, and watching her perky tits bounce, round butt jiggle is giving me serious goosebumps and tight pants syndrome. Nora Rose Eye Of The Beholder for Nude Muse They say beauty is in the eye of the beholder, but what about seduction? I would say that Nora Rose from Nude Muse is very seductive with her pure nude posing, and I gotta say, she has one of the most unique curvy bodies Ive ever seen. Big perky breasts, thick round hips, and an ass that wont quit. This is a list that is always updated with c

In [30]:
%cd /content/sample_data
FILE_DIR='./imgExtract'
if os.path.exists(FILE_DIR)==False:
  %mkdir imgExtract
  
# response = requests.get(site)

# soup = BeautifulSoup(html, 'html.parser')
img_tags = soup.find_all('img')

urls = [img['src'] for img in img_tags]

# FILE_DIR='./imgExtract/'
for url in urls:
    filename = re.search(r'/([\w_-]+[.](jpg|gif|png))$', url)
    if filename is not None:
        with open(FILE_DIR+'/'+filename.group(1), 'wb') as f:
            if 'http' not in url:
                # sometimes an image source can be relative 
                # if it is provide the base url which also happens 
                # to be the site variable atm. 
                url = '{}{}'.format(site, url)
            response = requests.get(url)
            f.write(response.content)  

/content/sample_data


In [31]:
df_list = []

try:
  for fil in os.listdir(FILE_DIR):
    filename=FILE_DIR+'/'+fil
    output=classifier.classify(filename)
    if (output[filename]['safe'].astype(float))  < (output[filename]['unsafe'].astype(float)):
      # print (output[filename]['safe'].astype(float))
      df_list.append(fil)
      # !rm fil
    # url_df = pd.concat(df_list, ignore_index=True).reset_index().drop('index',axis=1)
except:
  pass
if len(df_list)>0:
  print("The site contains unrestricted image!")
else:
  print("The site does not contain any unrestricted image!")

The site contains unrestricted image!


In [0]:
shutil.rmtree(FILE_DIR)

In [33]:
# learn_classify.predict(boilerpipe_sentences)
x=learn_classify.predict(boilerpipe_sentences)
y=pd.DataFrame(x[2],index=label_cols_classify)
# x[2].shape
y.nlargest(5,[0])

,0
Recreation,0.497899
Shopping,0.270266
Science,0.180146
Games,0.056721
Home,0.049648


In [34]:
x=learn_toxic.predict(boilerpipe_sentences)
y=pd.DataFrame(x[2],index=label_cols_toxic)
# x[2].shape
y.nlargest(5,[0])

,0
obscene,0.120532
toxic,0.056748
insult,0.000989
threat,0.000358
identity_hate,0.000274
